In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM, TextStreamer
import torch
import evaluate
from utils import save_response

# PHI3 SHORT STORY MODEL 

In [2]:
tokenizer_model = "unsloth/Phi-3-mini-4k-instruct"
model_id = "oztrkoguz/phi3_short_story_merged_bfloat16"
tokenizer = AutoTokenizer.from_pretrained(tokenizer_model)
model = AutoModelForCausalLM.from_pretrained(model_id).to("cuda")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [3]:
input_text = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
We have created a fitness app that helps users train for marathons. The app uses stories to help users train. The stories are generated based on user input/biometric data. 
The stories are meant to be played during a training session in order to help the user train for marathons.
You are a AI writer who's job is to create stories in various genres.
The genre and description will come by user input called Prompt.
The story should have embedded cues for user speed changes and breaks where approriate depending on the Training Style.

### Input:
Can you help me write a story with the following details:
Prompt: A action packed adventure story based off of Indiana Jones, include outrunning deadly traps, a chase scene with a treasure hunter and a villain, jungles and ancient ruins.
Distance: 13.1 miles
Average Walking Speed: 2.5 mph
Average Running Speed: 7.2 mph
Training Style: Half Marathon

### Response:
"""
print(input_text)


Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
We have created a fitness app that helps users train for marathons. The app uses stories to help users train. The stories are generated based on user input/biometric data. 
The stories are meant to be played during a training session in order to help the user train for marathons.
You are a AI writer who's job is to create stories in various genres.
The genre and description will come by user input called Prompt.
The story should have embedded cues for user speed changes and breaks where approriate depending on the Training Style.

### Input:
Can you help me write a story with the following details:
Prompt: A action packed adventure story based off of Indiana Jones, include outrunning deadly traps, a chase scene with a treasure hunter and a villain, jungles and ancient ruins.
Distance: 13.1 miles
Average Walking S

In [11]:

hyperparams = {
    "max_new_tokens": 1000, 
    # "temperature":0.1, 
    "do_sample": True,
    "repetition_penalty":5.0, 
    "no_repeat_ngram_size":2
}
encoded_input = tokenizer([input_text], return_tensors='pt').to("cuda")
streamer = TextStreamer(tokenizer)
response = model.generate(input_ids=encoded_input.input_ids, 
                        streamer=streamer, 
                        max_new_tokens=hyperparams["max_new_tokens"], 
                        # temperature=hyperparams["temperature"], 
                        do_sample=hyperparams["do_sample"],
                        repetition_penalty=hyperparams["repetition_penalty"], 
                        no_repeat_ngram_size=hyperparams["no_repeat_ngram_size"]).to("cuda")

<s> Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
We have created a fitness app that helps users train for marathons. The app uses stories to help users train. The stories are generated based on user input/biometric data. 
The stories are meant to be played during a training session in order to help the user train for marathons.
You are a AI writer who's job is to create stories in various genres.
The genre and description will come by user input called Prompt.
The story should have embedded cues for user speed changes and breaks where approriate depending on the Training Style.

### Input:
Can you help me write a story with the following details:
Prompt: A action packed adventure story based off of Indiana Jones, include outrunning deadly traps, a chase scene with a treasure hunter and a villain, jungles and ancient ruins.
Distance: 13.1 miles
Average Walki

In [12]:
output = tokenizer.decode(response[0], skip_special_tokens=True)
print(output)

Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
We have created a fitness app that helps users train for marathons. The app uses stories to help users train. The stories are generated based on user input/biometric data. 
The stories are meant to be played during a training session in order to help the user train for marathons.
You are a AI writer who's job is to create stories in various genres.
The genre and description will come by user input called Prompt.
The story should have embedded cues for user speed changes and breaks where approriate depending on the Training Style.

### Input:
Can you help me write a story with the following details:
Prompt: A action packed adventure story based off of Indiana Jones, include outrunning deadly traps, a chase scene with a treasure hunter and a villain, jungles and ancient ruins.
Distance: 13.1 miles
Average Walking S

In [13]:
save_response("PHI3-SHORT-STORY", hyperparams, output)

In [14]:
# perplexity = evaluate.load("perplexity", module_type="metric")
# input_texts = ["lorem ipsum", "Happy Birthday!", "Bienvenue"]
# results = perplexity.compute(model_id=model_id,
#                              add_start_token=False,
#                              predictions=input_texts, 
#                              device="cuda")
# print(list(results.keys()))

In [15]:
# print(round(results["mean_perplexity"], 2))